# Step 1 - Data Gathering

In [45]:
import numpy as np
import data_processor
from sklearn.neural_network import MLPClassifier
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler     # scaling transform
from sklearn.model_selection import train_test_split # validation tools
from collections import defaultdict

%matplotlib inline

seed = 1234

### Getting the Data

In [46]:
get_data_from_file = False  # faster when this is False
DATA_IMG_SIZE = (100, 100)

# get data from file
if (get_data_from_file):
    DATA_FILE = "data_images_100_100.txt"
    data = np.genfromtxt(DATA_FILE)
    X = data[:, :-1]
    y = data[:, -1]

# generate data here
if not get_data_from_file:
    X, y = data_processor.convert_images(DATA_IMG_SIZE, True)
    X = np.array(X)
    y = np.char.lower(np.array(y))


--- Done reading legend.csv ---
--- Done turning images into vectors ---


### Preprocessing the Data

In [47]:
from sklearn.preprocessing import LabelEncoder
import pandas as pd

# encode y (str -> int)
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)
label_mapping = pd.DataFrame({'class_label': label_encoder.classes_, 'numeric_label': np.arange(len(label_encoder.classes_))})
label_mapping.to_csv('label_mapping.csv', index=False)
num_classes = len(label_encoder.classes_)

# channels = 1 --> grayscale
original_height, original_width, channels = *DATA_IMG_SIZE, 1

# scale X
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
# reshape X
X_reshaped = X_scaled.reshape(-1, original_height, original_width, channels)

# split the training and testing data
X_tr, X_te, y_tr, y_te = train_test_split(X_reshaped, y_encoded, test_size=0.1, random_state=seed, shuffle=True)

# Step 2 - Building the Convolutional Neural Network

### Create and Test an Initial CNN

In [50]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd

# first 10000 of train for training, rest of train for validation
X_tr_train = X_tr[:10000, :]    # shape = (10000, 10000)
y_tr_train = y_tr[:10000]       # shape = (2321, 10000)
X_tr_val = X_tr[10000:, :]      # shape = (10000, )
y_tr_val = y_tr[10000:]         # sahpe = (2321, )

# create the CNN model
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(original_height, original_width, channels)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(num_classes, activation='softmax'))

# build the CNN model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# train the CNN model
model.fit(X_tr_train, y_tr_train, epochs=3, validation_data=(X_tr_val, y_tr_val))

# evalulate the CNN model
result = model.evaluate(X_te, y_te)
result_dict = dict(zip(model.metrics_names, result))
print(f"Model Result\n-----\n{result_dict}")

Epoch 1/3
313/313 [==============================] - 33s 102ms/step - loss: 0.7491 - accuracy: 0.7442 - val_loss: 0.5972 - val_accuracy: 0.7992
Epoch 2/3
313/313 [==============================] - 29s 94ms/step - loss: 0.5165 - accuracy: 0.8235 - val_loss: 0.5482 - val_accuracy: 0.8104
Epoch 3/3
43/43 [==============================] - 1s 22ms/step - loss: 0.4862 - accuracy: 0.8291
Model Result
-----
{'loss': 0.48619744181632996, 'accuracy': 0.8290722966194153}
